In [1]:
%matplotlib inline

# 신경망 모델 구성하기

신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있다.
[`torch.nn`](<https://pytorch.org/docs/stable/nn.html>) 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공한다.
PyTorch의 모든 모듈은 [`nn.Module`](<https://pytorch.org/docs/stable/generated/torch.nn.Module.html>)의 하위 클래스(subclass)
이다. 신경망은 그 자체로 다른 모듈 혹은 계층(layer)들로 구성된 하나의 모듈이다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있게 한다.

이제 FashionMNIST 데이터셋의 이미지들을 분류하는 신경망을 구성해보자.


In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

학습을 위한 장치 얻기
------------------------------------------------------------------------------------------

가능하다면 GPU와 같은 하드웨어 가속기에서 모델을 학습하는 것이 효율적이다.
[`torch.cuda`](<https://pytorch.org/docs/stable/notes/cuda.html>)를 사용할 수 있는지
확인하고 그렇지 않으면 CPU를 계속 사용한다.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


클래스 정의하기
------------------------------------------------------------------------------------------

신경망 모델을 ``nn.Module``의 하위클래스로 정의하고, ``__init__``에서 신경망 계층들을 초기화한다.
``nn.Module``을 상속받은 모든 클래스는 ``forward`` 메소드에 입력 데이터에 대한 연산들을 구현한다.



In [ ]:
img_height = 28     # 이미지의 높이
img_width = 28      # 이미지의 너비
num_channels = 1    # 흑백 이미지이므로 1
num_classes = 10    # 분류할 이미지의 클래스는 10가지

In [ ]:
class NeuralNetwork(nn.Module):
    #네트워크의 계층을 정의 먼저 입력 텐서를 1D 텐서로 평면화하는 nn.flatten인스턴스 생성
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        #그 다음 선형 및 활성화 계층을 보관하기 위해 nn.sequential 컨테이너 생성
        #컨테이너는 세 개의 선형레이어 정의
        self.linear_relu_stack = nn.Sequential(
            #(img_heightimg_widthnum_channels, 512) 형태의 입력 텐서를 취하고 선형 변환을 적용하여 512 형태의 출력 텐서 생성
            nn.Linear(img_height*img_width*num_channels, 512),
            #relu 활성화 함수를 첫번째 계층의 출력에 적용
            nn.ReLU(),
            #512 형태의 입력 텐서를 취하고 선형 변환을 적용하여 num_classes 형태의 출력 텐서를 생성
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes),
        )
    #입력 텐서 x가 flatten 계층을 사용하여 평면화되고 linear _relu_stack에 정의된 선형 및 활성화 계층을 통과
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        #결과 텐서는 각 입력 샘플에 대해 예측된 클래스 점수를 나타내는 logits로 변환
        return logits

거의 유사한 구조의 네트워크를 약간 다르게 정의해보았다.

In [ ]:
#3개의 완전히 연결된(선형)계층이 있으며 이들 사이에 ReLU활성화 기능을 사용 
#ReLU 단순성, 계산 효율성 및 기울기 소실 문제를 방지하는 기능 rectified linear unit
class NeuralNetwork2(nn.Module):
    #생성자 init에서 nn.Linear 및 nn.ReLU모듈사용하여 레이어를 설정하고 클래스의 인스턴수 변수로 저장하여 모델을 정의
    def __init__(self):
        super(NeuralNetwork2, self).__init__()
        #fc1 첫번째 완전 연결 계층은 입력 특성이 이미지의 높이, 너비 및 채널 수의 곱, 출력 특성 1024개
        self.fc1 = nn.Linear(img_height*img_width*num_channels, 1024) 
        #fc2 두번째 완전 연결 계층 1024개의 입력 특성과 512개의 출력 특성
        self.fc2 = nn.Linear(1024, 512)
        #512개의 입력특성과 분류를 위한 클래스 수와 같은 출력 특성
        self.fc3 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()
    #forward 방법에서는 입력을 view를 사용하여 평면화하여 2D 이미지 텐서에서 1D텐서로 변환
    #평평한 텐서는 그 사이에 ReLU 활성화가 있는 3개의 선형 레이어를 통과하고 최종 출력이 반환
    #신경망의 순전파 정의
    def forward(self, x):
      #1D텐서로 변환
        x = x.view(x.size(0), -1)     # flatten the input image
        #평평한 텐서는 fc1을 통고한 다음 relu 활성화 함수를 거침
        out = self.relu(self.fc1(x))
        #이전 단계의 출력이 두번째 fc2 거친 다음 다시 relu 활성화 거침
        out = self.relu(self.fc2(out))
        #fc3거쳐 최종 출력 생성
        out = self.fc3(out)
        return out

``NeuralNetwork``의 인스턴스(instance)를 생성하고 이를 ``device``로 이동한 뒤,
구조(structure)를 출력한다.



In [ ]:
model = NeuralNetwork().to(device)
print(model)
#첫번째 평평한 이미지 텐서 입력 사용하고 512 차원의 텐서를 출력
#두번째 이 텐서를 가져와 다시 512 차원의 텐서 출력
#3번째 이 텐서를 가져와 num_classes 차원의 텐서를 (출력 모델이 예측해야하는 클래스의 수)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델에 입력 데이터를 전달하면 모델의 ``forward`` 메서드가 자동으로 실행된다. ``model.forward()``를 사용자가 직접 호출하지는 않는다.

입력 데이터에 대해 모델을 실행하면 각 분류(class)에 대한 원시(raw) 예측값이 저장된 10-차원 텐서가 반환된다. 이 값을 각 분류에 속할 확률(각각은 0에서 1사이의 실수이고 합이 1이 되는 실수들)로 변환하고 싶다면 ``nn.Softmax`` 모듈을 적용하면 된다. 

In [ ]:
#2개의 그레이스케일 이미지를 나타내는 모양 (2,1,28,28)의 임의 텐서 x를 생성
#그 다음 모델의 원시 출력인 로직을 얻기 위해 신경망 모델인 모델을 통해 x전달 로직의 모양은 (2,classes)
#(이미지수,색상채널수,이미지 높이,너비)
X = torch.rand(2, 1, 28, 28, device=device)  # 2 random grayscale images
#입력데이터x에 대한 모델의 예측 출력값
logits = model(X)
print('logits: {}'.format(logits))
#소프트맥스 함수 nn.Softmax(dim=1)를 사용하여 로직 텐서의 두번째 차원을 따라 적용되어 (2,num_classes)의 예측 확률 텐서를 얻음
#더 정교화된 얘측값 제공
pred_probab = nn.Softmax(dim=1)(logits)
print('pred_probab: {}'.format(pred_probab))
#예측 확률이 가장 높은 클래스를 기반로 2개의 이미지 각각에 대한 예측 클래스 인덱스 얻어 y_pred에 저장
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

logits: tensor([[ 0.0698, -0.0292, -0.0986,  0.0445,  0.0170,  0.0153,  0.0006, -0.0215,
          0.0360, -0.0048],
        [ 0.0596, -0.0491, -0.0849,  0.0363,  0.0162, -0.0029, -0.0198, -0.0381,
          0.1013,  0.0874]], device='cuda:0', grad_fn=<AddmmBackward0>)
pred_probab: tensor([[0.1068, 0.0967, 0.0903, 0.1041, 0.1013, 0.1012, 0.0997, 0.0975, 0.1033,
         0.0991],
        [0.1048, 0.0941, 0.0907, 0.1024, 0.1004, 0.0985, 0.0968, 0.0951, 0.1093,
         0.1078]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([0, 8], device='cuda:0')


------------------------------------------------------------------------------------------




모델 계층(Layer)
------------------------------------------------------------------------------------------

FashionMNIST 모델의 계층들을 살펴보자. 이를 설명하기 위해, 28x28 크기의 랜덤 이미지 3개로 구성된
미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아본다.



In [ ]:
input_images = torch.rand(3, 1, 28, 28) # 3 random grayscale images
#배치에 3개의 이미지 샘플이 있고 각각 1개의 채널이 있고 각 이미지의 높이와 너비가 28픽셀
#두번째는 색상 채널인데 회색조이므로 하나만 있으면 된다
print(input_images)
print(input_images.size())

tensor([[[[0.5899, 0.1377, 0.5635,  ..., 0.7419, 0.5389, 0.1329],
          [0.4346, 0.5824, 0.1957,  ..., 0.9177, 0.9711, 0.8584],
          [0.5505, 0.7385, 0.4338,  ..., 0.3232, 0.7114, 0.1898],
          ...,
          [0.2887, 0.1678, 0.9462,  ..., 0.7314, 0.4096, 0.9896],
          [0.6946, 0.0042, 0.1949,  ..., 0.6815, 0.3720, 0.6154],
          [0.2296, 0.9481, 0.8549,  ..., 0.7209, 0.1182, 0.0214]]],


        [[[0.2177, 0.5515, 0.6387,  ..., 0.9155, 0.4830, 0.5330],
          [0.8900, 0.0887, 0.8392,  ..., 0.7162, 0.9012, 0.3201],
          [0.7249, 0.7423, 0.7093,  ..., 0.1522, 0.4259, 0.6443],
          ...,
          [0.5613, 0.9525, 0.0628,  ..., 0.7070, 0.5746, 0.8042],
          [0.9412, 0.5226, 0.9981,  ..., 0.3875, 0.2468, 0.2501],
          [0.4412, 0.7136, 0.2252,  ..., 0.2590, 0.7530, 0.8188]]],


        [[[0.0178, 0.8296, 0.6243,  ..., 0.2192, 0.6648, 0.8247],
          [0.3952, 0.3111, 0.8062,  ..., 0.1829, 0.2982, 0.5585],
          [0.1321, 0.4822, 0.0016,  ..

#### `nn.Flatten`

[`nn.Flatten`](<https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html>) 계층은
각 28x28의 2D 이미지를 784 픽셀 값을 갖는 1차원 배열로 변환한다. dim=0의 미니배치 차원은 유지된다.



배치(batch) 처리를 하는 이유는 여러 가지가 있습니다. 가장 큰 이유는 GPU의 병렬 처리 기능을 활용하여 추론 속도를 높이기 위함입니다. GPU는 단일 연산에 대해 매우 높은 성능을 보이는데, 이를 이용하여 한 번에 여러 개의 이미지를 처리하면 전체 추론 시간을 대폭 줄일 수 있습니다.

또한, 배치 처리를 함으로써 효율적인 데이터 로딩이 가능해집니다. 이미지 파일을 읽어들이는 작업은 I/O 연산으로 인해 매우 느리기 때문에, 한 번에 여러 개의 이미지를 로딩하여 메모리에 올려두면 I/O 연산을 줄일 수 있습니다.

마지막으로, 배치 처리를 함으로써 모델의 일반화 성능을 높일 수 있습니다. 이는 데이터셋에서 임의로 추출된 배치가 전체 데이터셋을 대표하는 정보를 담고 있을 가능성이 높기 때문입니다. 따라서 배치를 이용하여 모델을 학습하면, 전체 데이터셋에 대한 일반화 성능이 향상됩니다.

따라서, 배치 처리는 딥러닝 모델의 추론 및 학습에서 중요한 역할을 합니다.

In [ ]:
#크기가 (3,1,28,28)인 텐서 input_images에 nn.Flatten() 작업을 적용하여 각각 임지를 나타냄
#nn.Flatten()작업은 각 이미지의 모든 픽셀 값을 연결하여 input_impages의 각 이미지를 1차원 텐서로 병합한다.
#결과 flat_image 텐서의 차원은 (3,784)이다. 여기서 3은 배치크기(이미지수)이고 784는 평면화 후 각 이미지의 총 픽셀 수
flatten = nn.Flatten()
flat_image = flatten(input_images)
print(flat_image.size())

torch.Size([3, 784])


#### `nn.Linear`

[선형 계층](<https://pytorch.org/docs/stable/generated/torch.nn.Linear.html>)은 저장된 가중치(weight)와
편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈이다.

In [ ]:
#선형 레이어 layer1정의 후 이 레이어를 평면화된 입력 이미지 flat_image에 
#적용 크기가 20인 특징 벡터로 표현하려면 평면화
#그 결과 크기가 3*20인 텐서가 생성 
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())
#배치의 각 이미지에 대한 20개의 숨겨진 단위 값을 포함하는 결과 텐서 출력
#20개의 특징을 구함
#cnn을 통해 모델 내부에서 자동으로 학습
print(hidden1)

torch.Size([3, 20])
tensor([[-0.5696, -0.0971,  0.4020,  0.1580,  0.5307,  0.1039, -0.5530, -0.2462,
          0.0940, -0.5691,  0.0738, -0.1913, -0.0880,  0.2391, -0.1928, -0.3406,
          0.7934,  0.2581, -0.1567, -0.0165],
        [-0.2834,  0.2402,  0.3911,  0.0409, -0.0400,  0.2782, -0.4495, -0.3883,
         -0.1695,  0.1305,  0.1453, -0.1059, -0.4036,  0.1702, -0.1996, -0.1264,
          0.4059,  0.2540, -0.2857, -0.0910],
        [-0.3287,  0.0295,  0.2424, -0.0582,  0.0675, -0.1619, -0.6016, -0.2853,
         -0.1620, -0.4376, -0.2458,  0.3466, -0.2085,  0.1128, -0.1083,  0.1271,
          0.2557,  0.0282, -0.0290, -0.3726]], grad_fn=<AddmmBackward0>)


#### `nn.ReLU`

비선형 활성화(activation)는 모델의 입력과 출력 사이에 비선형적인 복잡한 관계(mapping)를 만든다.
비선형 활성화는 선형 변환 후에 적용되어 *비선형성(nonlinearity)* 을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다.

이 모델에서는 [`nn.ReLU`](<https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html>)를 사용하지만, 다양한 다른 활성화 함수를 사용할 수도 있다. 

**Note:** $ReLu(x) = \max(0, x)$이다.



In [ ]:
#relu 활성화 함수를 layer1의 출력에 적용하여 신경망에 비선형성을 도입
#relue 함수는 0과 입력  텐서 사이의 요소별 최대값을 반환
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}") 
#relue활성화 함수는 음수 값을 0으로 설정, 양수값은 유지
#이렇게 하면 활성화 함수의 출력이 항상 음수가 아니게 되고 신경망에 비선형성이 도입됨

Before ReLU: tensor([[-0.5696, -0.0971,  0.4020,  0.1580,  0.5307,  0.1039, -0.5530, -0.2462,
          0.0940, -0.5691,  0.0738, -0.1913, -0.0880,  0.2391, -0.1928, -0.3406,
          0.7934,  0.2581, -0.1567, -0.0165],
        [-0.2834,  0.2402,  0.3911,  0.0409, -0.0400,  0.2782, -0.4495, -0.3883,
         -0.1695,  0.1305,  0.1453, -0.1059, -0.4036,  0.1702, -0.1996, -0.1264,
          0.4059,  0.2540, -0.2857, -0.0910],
        [-0.3287,  0.0295,  0.2424, -0.0582,  0.0675, -0.1619, -0.6016, -0.2853,
         -0.1620, -0.4376, -0.2458,  0.3466, -0.2085,  0.1128, -0.1083,  0.1271,
          0.2557,  0.0282, -0.0290, -0.3726]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.4020, 0.1580, 0.5307, 0.1039, 0.0000, 0.0000, 0.0940,
         0.0000, 0.0738, 0.0000, 0.0000, 0.2391, 0.0000, 0.0000, 0.7934, 0.2581,
         0.0000, 0.0000],
        [0.0000, 0.2402, 0.3911, 0.0409, 0.0000, 0.2782, 0.0000, 0.0000, 0.0000,
         0.1305, 0.1453, 0.0000, 0.0000, 0.1702, 0.00

#### `nn.Sequential`

[`nn.Sequential`](<https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html>)은 순서를 갖는
모듈의 컨테이너이다. 데이터는 모듈들을 나열된 순서대로 통과한다. 순차 컨테이너(sequential container)를 사용하여 아래의 ``seq_modules``와 같은 신경망을 빠르게 만들 수 있다.



In [ ]:
seq_modules = nn.Sequential(
    flatten, #28*28을 784로 평평하게
    layer1, #입력텐서에서 선형 변환을 수행하는 nn.linear의 인스턴스
    nn.ReLU(), #layer1의 출력에 relu함수를 요소별로 적용하는 활성화 함수
    nn.Linear(20, 10)#네트워크의 최종 출력을 생성하는 in_feature=20 및 out_feature=10인 선형레이어
)
#무작위 이미지 배치(28*28인 3대) 만들고 네트워크에 전달하여 (3,10) 형태의 텐서인 출력 로직을 얻음
input_images = torch.rand(3, 1, 28, 28)
logits = seq_modules(input_images)
logits

tensor([[-0.0333,  0.2017,  0.2365, -0.1200,  0.0542,  0.2103,  0.0135, -0.3470,
         -0.2061,  0.0491],
        [ 0.0039,  0.1499,  0.2371, -0.1485,  0.0523,  0.3345,  0.0496, -0.3788,
         -0.2201,  0.0369],
        [ 0.0689,  0.1479,  0.2206, -0.1252,  0.0388,  0.2189, -0.0534, -0.2929,
         -0.1407,  0.0749]], grad_fn=<AddmmBackward0>)

#### `nn.Softmax`

신경망의 마지막 선형 계층은 [`nn.Softmax`](<https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html>) 모듈에 전달될
[-\infty, \infty] 범위의 원시 값(raw value)인 `logits`를 반환한다. `nn.Softmax` 계층은 logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록
[0, 1] 범위로 비례하여 조정(scale)한다. ``dim`` 매개변수는 값의 합이 1이 되는 차원을 나타낸다.



In [ ]:
#logits를 각 클래스의 예측 확률로 변환하는 softmax 함수를 logits 텐서에 적용
#nn.Softmax(dim=1) 함수는 softmax 작업이 수행될 차원을 지정하는 dim인수 사용 텐서의 2번째 차원을 따라 적용됨
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
#결과 텐서 pred_probab에는 입력 이미지에 대한 각 클래스의 예측 확률이 포함되며 
#각 행은 하나의 입력 이미지에 해당하고 각 열은 하나의 클래스에 해당
pred_probab

**Note:** `SoftMax`함수 $\sigma : R^K \longrightarrow (0, 1)^K$의 정의: $\sigma(z)_i = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}}$

모델 매개변수
------------------------------------------------------------------------------------------

신경망에서 많은 계층들은 매개변수화(parameterize)되어 있다. 즉, 학습 중에 최적화되는 가중치(weight)와 편향(bias)을 가진다. ``nn.Module``을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 ``parameters()`` 및
``named_parameters()`` 메소드로 모든 매개변수에 접근할 수 있게 된다.

이 예제에서는 각 매개변수들을 순회하며(iterate), 매개변수의 크기와 값을 출력한다.




In [2]:
#모델의 구조와 해당 매개변수의 세부사항을 인쇄
print(f"Model structure: {model}\n\n")

#모델의 구조 표시 모델 객체의 named_parameters는 모델의 모든 매개변수 반복하고 이름,크기 및 처음 두값 인쇄하는데 사용
#이는 매개변수 값이 올바르게 초기화 되었는지 확인하기 위해 수행
for name, param in model.named_parameters(): 
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")
    # [:2] 슬라이스는 커질 수 있는 전체 텐서를 인쇄하지 않도록 각 텐서의 처음 두 값만 인쇄하는 데 사용됩니다.
    #fc1 레이어의 크기, 해당 레이어의 가중치 행렬 및 편향 벡터의 처음 두 값 인쇄
    #fc1.weight, fc1.bias, fc2.weight, fc2.bias, fc3.weight, fc3.bias

NameError: ignored

------------------------------------------------------------------------------------------




더 읽어보기
------------------------------------------------------------------------------------------
- [`torch.nn API`](<https://pytorch.org/docs/stable/nn.html>)

